In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext notexbook
%texify

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
sys.setrecursionlimit(10000)
%matplotlib qt5

In [3]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record

In [4]:
param_file = '../../cellbgnet/utils/reference_files/reference_membrane.yaml'
param = cellbgnet.utils.param_io.ParamHandling().load_params(param_file)

In [5]:
psf_params = param.PSF.to_dict()
simulation_params = param.Simulation.to_dict()
hardware_params = param.Hardware.to_dict()
eval_params = param.Evaluation.to_dict()
train_size = simulation_params['train_size']

In [6]:
model = cellbgnet.model.CellBGModel(param)

training sliding windows on camera chip:
Area num: 0, field_xy: [0, 127, 0, 127]
Area num: 1, field_xy: [114, 241, 0, 127]
Area num: 2, field_xy: [228, 355, 0, 127]
Area num: 3, field_xy: [342, 469, 0, 127]
Area num: 4, field_xy: [456, 583, 0, 127]
Area num: 5, field_xy: [570, 697, 0, 127]
Area num: 6, field_xy: [684, 811, 0, 127]
Area num: 7, field_xy: [798, 925, 0, 127]
Area num: 8, field_xy: [912, 1039, 0, 127]
Area num: 9, field_xy: [1026, 1153, 0, 127]
Area num: 10, field_xy: [1140, 1267, 0, 127]
Area num: 11, field_xy: [1174, 1301, 0, 127]
Area num: 12, field_xy: [0, 127, 114, 241]
Area num: 13, field_xy: [114, 241, 114, 241]
Area num: 14, field_xy: [228, 355, 114, 241]
Area num: 15, field_xy: [342, 469, 114, 241]
Area num: 16, field_xy: [456, 583, 114, 241]
Area num: 17, field_xy: [570, 697, 114, 241]
Area num: 18, field_xy: [684, 811, 114, 241]
Area num: 19, field_xy: [798, 925, 114, 241]
Area num: 20, field_xy: [912, 1039, 114, 241]
Area num: 21, field_xy: [1026, 1153, 114, 24

In [7]:
model.init_recorder()
model.init_eval_data()

Eval image generation: 100%|████████████████████| 30/30 [00:05<00:00,  5.01it/s]


eval images shape: (30, 1041, 1302) contain 11839 molecules,


In [8]:
model.evaluation_params['eval_imgs'].shape

(30, 1041, 1302)

In [9]:
model.simulation_params

{'train_size': 128,
 'train_type': 'uniform',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 30,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.15,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 220.0,
 'bg_width': 40,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [10]:
start_time = time.time()
model.fit(batch_size=32, max_iters=20000, print_output=True, print_freq=100)
duration = time.time() - start_time

Started training .... 
 || Cost: 15267.954 || Time Upd.: 680.1 ms  || BatchNr.: 100
 || Cost: 2275.841 || Time Upd.: 674.7 ms  || BatchNr.: 200
 || Cost: 644.854 || Time Upd.: 696.7 ms  || BatchNr.: 300
 || Cost: 572.400 || Time Upd.: 690.2 ms  || BatchNr.: 400
 || Cost: 431.544 || Time Upd.: 679.9 ms  || BatchNr.: 500
 || Cost: 337.059 || Time Upd.: 683.9 ms  || BatchNr.: 600
 || Cost: 335.366 || Time Upd.: 690.0 ms  || BatchNr.: 700
 || Cost: 288.303 || Time Upd.: 682.8 ms  || BatchNr.: 800
 || Cost: 275.591 || Time Upd.: 669.3 ms  || BatchNr.: 900
 || Cost: 255.124 || Time Upd.: 639.4 ms  || BatchNr.: 1000
processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 8700
JoR: 1.948 || Eff_3d: 48.744 || Jaccard: 0.738 || Factor: 412.286 || RMSE_lat: 37.862 || RMSE_ax: 99.960 || RMSE_

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 10873
JoR: 2.562 || Eff_3d: 60.306 || Jaccard: 0.928 || Factor: 428.791 || RMSE_lat: 36.219 || RMSE_ax: 83.678 || RMSE_x: 21.623 || RMSE_y: 29.056 || Cost: 193.359 || Recall: 0.929 || Precision: 0.998 || BatchNr.: 2200 || Time Upd.: 630.7 ms 
 || Count Cost: 123.9942 || Localization Cost: 125.8895 || Bg Cost: 0.2024 || P_locs Cost: 105.9184saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 60.306 || 36.219 || 83.678 || 2200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 10980
JoR: 2.854 || Eff_3d: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11360
JoR: 2.949 || Eff_3d: 62.799 || Jaccard: 0.963 || Factor: 489.397 || RMSE_lat: 32.657 || RMSE_ax: 82.737 || RMSE_x: 23.372 || RMSE_y: 22.808 || Cost: 157.246 || Recall: 0.968 || Precision: 0.995 || BatchNr.: 3400 || Time Upd.: 649.3 ms 
 || Count Cost: 116.8805 || Localization Cost: 99.5427 || Bg Cost: 0.1223 || P_locs Cost: 98.1613saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 62.799 || 32.657 || 82.737 || 3400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 10442
JoR: 3.234 || Eff_3d: 65

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11142
JoR: 3.185 || Eff_3d: 66.424 || Jaccard: 0.950 || Factor: 399.422 || RMSE_lat: 29.839 || RMSE_ax: 73.134 || RMSE_x: 20.903 || RMSE_y: 21.293 || Cost: 139.018 || Recall: 0.952 || Precision: 0.998 || BatchNr.: 4600 || Time Upd.: 708.0 ms 
 || Count Cost: 109.6135 || Localization Cost: 85.3671 || Bg Cost: 0.1058 || P_locs Cost: 93.0446saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 66.424 || 29.839 || 73.134 || 4600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11138
JoR: 3.208 || Eff_3d: 66

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11061
JoR: 3.154 || Eff_3d: 63.290 || Jaccard: 0.941 || Factor: 436.255 || RMSE_lat: 29.848 || RMSE_ax: 85.193 || RMSE_x: 21.102 || RMSE_y: 21.109 || Cost: 138.745 || Recall: 0.944 || Precision: 0.997 || BatchNr.: 5800 || Time Upd.: 649.3 ms 
 || Count Cost: 118.9819 || Localization Cost: 83.0284 || Bg Cost: 0.1101 || P_locs Cost: 92.4398saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 63.290 || 29.848 || 85.193 || 5800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11237
JoR: 3.197 || Eff_3d: 66

...

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11345
JoR: 3.275 || Eff_3d: 68.070 || Jaccard: 0.967 || Factor: 412.657 || RMSE_lat: 29.519 || RMSE_ax: 67.983 || RMSE_x: 20.664 || RMSE_y: 21.080 || Cost: 108.152 || Recall: 0.969 || Precision: 0.998 || BatchNr.: 14200 || Time Upd.: 654.2 ms 
 || Count Cost: 107.4167 || Localization Cost: 59.5173 || Bg Cost: 0.0698 || P_locs Cost: 86.2744saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 68.070 || 29.519 || 67.983 || 14200


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11424
JoR: 3.244 || Eff_3d: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11343
JoR: 3.346 || Eff_3d: 68.704 || Jaccard: 0.968 || Factor: 410.554 || RMSE_lat: 28.914 || RMSE_ax: 66.679 || RMSE_x: 20.260 || RMSE_y: 20.628 || Cost: 111.094 || Recall: 0.969 || Precision: 0.998 || BatchNr.: 15400 || Time Upd.: 686.2 ms 
 || Count Cost: 108.2956 || Localization Cost: 61.6284 || Bg Cost: 0.0744 || P_locs Cost: 87.4028saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 68.704 || 28.914 || 66.679 || 15400


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11417
JoR: 3.261 || Eff_3d: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11386
JoR: 3.262 || Eff_3d: 68.390 || Jaccard: 0.967 || Factor: 421.518 || RMSE_lat: 29.649 || RMSE_ax: 66.455 || RMSE_x: 20.705 || RMSE_y: 21.222 || Cost: 107.200 || Recall: 0.971 || Precision: 0.996 || BatchNr.: 16600 || Time Upd.: 639.3 ms 
 || Count Cost: 107.8136 || Localization Cost: 58.8482 || Bg Cost: 0.0634 || P_locs Cost: 86.8653saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 68.390 || 29.649 || 66.455 || 16600


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11388
JoR: 3.258 || Eff_3d: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11342
JoR: 3.295 || Eff_3d: 68.158 || Jaccard: 0.964 || Factor: 410.525 || RMSE_lat: 29.261 || RMSE_ax: 68.037 || RMSE_x: 20.478 || RMSE_y: 20.902 || Cost: 106.900 || Recall: 0.968 || Precision: 0.996 || BatchNr.: 17800 || Time Upd.: 636.2 ms 
 || Count Cost: 108.6809 || Localization Cost: 58.3489 || Bg Cost: 0.0657 || P_locs Cost: 86.2347saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 68.158 || 29.261 || 68.037 || 17800


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11430
JoR: 3.297 || Eff_3d: 

processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11216
JoR: 3.381 || Eff_3d: 69.381 || Jaccard: 0.956 || Factor: 387.814 || RMSE_lat: 28.268 || RMSE_ax: 64.654 || RMSE_x: 19.697 || RMSE_y: 20.276 || Cost: 105.295 || Recall: 0.958 || Precision: 0.998 || BatchNr.: 19000 || Time Upd.: 703.6 ms 
 || Count Cost: 108.9048 || Localization Cost: 56.7955 || Bg Cost: 0.0639 || P_locs Cost: 86.4348saving this model, eff_3d, rmse_lat, rmse_ax and BatchNr are : 69.381 || 28.268 || 64.654 || 19000


processing area:99/99, input field_xy:[1258 1301 1001 1040], use_coordconv:True, retain locs in area:[1278, 1301, 1021, 1040]

evaluation on 30 images, contain ground truth: 11839FOV: x=[0, 84630] y=[0, 67665]
after FOV and border segmentation,truth: 11679 ,preds: 11409
JoR: 3.274 || Eff_3d: 

In [12]:
1+1

2